In [ ]:
# 필요한 라이브러리 설치
!pip install transformers
!pip install torch
!pip install transformers torch
!pip install konlpy scikit-learn
!pip install kobert-transformers
!pip install bert-extractive-summarizer
!pip install transformers --upgrade
!pip install scikit-learn

# 구글 드라이브 마운트
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from google.colab import drive
from transformers import T5ForConditionalGeneration, T5Tokenizer, Trainer, TrainingArguments
from konlpy.tag import Okt
import pandas as pd
import re
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer
from transformers import BertForSequenceClassification



In [ ]:
pip install --pre torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/nightly/cu122


In [ ]:
!pip install accelerate -U

# 또는 transformers[torch]를 설치하는 경우 (이 명령은 필요한 모든 의존성을 설치합니다)
!pip install transformers[torch]

In [ ]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

In [ ]:
# 데이터 로드 및 전처리를 위한 함수 정의
def load_data(file_paths):
    dataframes = []
    for file_path in file_paths:
        df = pd.read_csv(file_path)
        dataframes.append(df)
    return pd.concat(dataframes, ignore_index=True)

def preprocess_text(text, stopwords):
    text = re.sub(r'[^가-힣\s]', '', text)  # 특수문자 제거, 한글만 남김
    okt = Okt()
    tokens = okt.morphs(text, stem=True)  # 형태소 분석
    tokens = [token for token in tokens if token not in stopwords]
    return ' '.join(tokens)

# 데이터셋 클래스 정의
class T5FineTuningDataset(Dataset):
    def __init__(self, tokenizer, data, source_max_token_len=512, target_max_token_len=128):
        self.tokenizer = tokenizer
        self.data = data
        self.source_max_token_len = source_max_token_len
        self.target_max_token_len = target_max_token_len

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        data_row = self.data.iloc[index]

        source_encoding = tokenizer(
            preprocess_text(data_row['content'], stopwords),
            max_length=self.source_max_token_len,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            add_special_tokens=True,
            return_tensors='pt'
        )

        target_encoding = tokenizer(
            data_row['label'],
            max_length=self.target_max_token_len,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            add_special_tokens=True,
            return_tensors='pt'
        )

        labels = target_encoding['input_ids']
        labels[labels == 0] = -100  # T5에서 padding token은 무시하기 위해 -100으로 설정

        return dict(
            input_ids=source_encoding['input_ids'].flatten(),
            attention_mask=source_encoding['attention_mask'].flatten(),
            labels=labels.flatten()
        )

# 파일 경로 설정
file_paths = [
    '/content/drive/MyDrive/fine/combined_text_with_labels.csv',
    '/content/drive/MyDrive/fine/combined_text_with_labels2.csv',
    '/content/drive/MyDrive/fine/combined_text_with_labels3.csv',
    '/content/drive/MyDrive/fine/combined_text_with_labels4.csv'
    # 다른 파일 경로를 추가하세요.
]

# 데이터 로드
data = load_data(file_paths)

# 불용어 설정
stopwords = set(["의", "가", "이", "은", "들", "는", "좀", "잘", "걍", "과", "도", "를", "으로", "자", "에", "와", "한", "하다"])

# 토크나이저 및 모델 로드
tokenizer = T5Tokenizer.from_pretrained('digit82/kolang-t5-base')
model = T5ForConditionalGeneration.from_pretrained('digit82/kolang-t5-base')

# 데이터셋 준비
dataset = T5FineTuningDataset(tokenizer, data)

# 훈련 설정
training_args = TrainingArguments(
    output_dir='/content/drive/MyDrive/Model/trained_model',
    num_train_epochs=3,  # 적절한 에포크로 조정하세요.
    per_device_train_batch_size=8,  # 하드웨어에 맞게 조정하세요.
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='/content/drive/MyDrive/Model/logs',
    logging_steps=10,
)

# 트레이너 초기화 및 훈련 시작
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
)

trainer.train()




In [ ]:
import torch
print(torch.__version__)


In [ ]:
print(torch.version.cuda)


In [ ]:
!nvcc --version

In [ ]:
!pip install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu113


In [ ]:
# 데이터 로드 및 전처리를 위한 함수 정의
def load_data(file_paths):
    dataframes = []
    for file_path in file_paths:
        df = pd.read_csv(file_path)
        dataframes.append(df)
    return pd.concat(dataframes, ignore_index=True)

def preprocess_text(text, stopwords):
    text = re.sub(r'[^가-힣\s]', '', text)  # 특수문자 제거, 한글만 남김
    okt = Okt()
    tokens = okt.morphs(text, stem=True)  # 형태소 분석
    tokens = [token for token in tokens if token not in stopwords]
    return ' '.join(tokens)

# 데이터셋 클래스 정의
class KoBertDataset(Dataset):
    def __init__(self, tokenizer, data, max_token_len=256):
        self.tokenizer = tokenizer
        self.data = data
        self.max_token_len = max_token_len

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        data_row = self.data.iloc[index]

        encoding = tokenizer(
            preprocess_text(data_row['content'], stopwords),
            add_special_tokens=True,
            max_length=self.max_token_len,
            return_token_type_ids=False,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt',
        )

        return dict(
            input_ids=encoding['input_ids'].flatten(),
            attention_mask=encoding['attention_mask'].flatten(),
            labels=torch.tensor(data_row['label'])
        )

# 파일 경로 설정
file_paths = [
    '/content/drive/MyDrive/fine/combined_text_with_labels.csv',
    '/content/drive/MyDrive/fine/combined_text_with_labels2.csv',
    '/content/drive/MyDrive/fine/combined_text_with_labels3.csv',
    '/content/drive/MyDrive/fine/combined_text_with_labels4.csv'

]

# 데이터 로드
data = load_data(file_paths)

# 불용어 설정
stopwords = set(["의", "가", "이", "은", "들", "는", "좀", "잘", "걍", "과", "도", "를", "으로", "자", "에", "와", "한", "하다"])

# 토크나이저 및 모델 로드
tokenizer = BertTokenizer.from_pretrained('monologg/kobert')
model = BertForSequenceClassification.from_pretrained('monologg/kobert', num_labels=2)  # num_labels는 데이터셋의 레이블 수에 맞춰 조정

# 데이터셋 준비
dataset = KoBertDataset(tokenizer, data)

# 훈련 설정
training_args = TrainingArguments(
    output_dir='/content/drive/MyDrive/Model/trained_model',
    num_train_epochs=3,  # 적절한 에포크로 조정하세요.
    per_device_train_batch_size=8,  # 하드웨어에 맞게 조정하세요.
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='/content/drive/MyDrive/Model/logs',
    logging_steps=10,
)

# 트레이너 초기화 및 훈련 시작
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
)

trainer.train()

In [ ]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"